In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium
pip install pathos

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import os
import progressbar
from more_itertools import chunked


In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1. 크롤링

In [ ]:
# 특정 사이트 크롤링
def get_video_list(url):
  nsl = url
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')        # Head-less 설정
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(nsl)
  time.sleep(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  list_video = soup.select('a.VideoBoxLinkWrapper_wrap_GLkZS')

  video_list = []
  replay_video_list = []
  for i in list_video:
      temp_url = i.get('href')
      video_list.append(temp_url)
      
      if 'replay' in temp_url:
          replay_video_list.append(temp_url)

  return replay_video_list

In [ ]:
my_list = get_video_list('https://shoppinglive.naver.com/channels/54565')
result = list(chunked(my_list, 5))
len(result)

2

In [ ]:
# 크롤러 생성

class MyCrawler:
    def __init__(self, numprocess, webdriverpath, path_save):
        self.n_process = numprocess
        self.pool = ProcessPool(nodes=self.n_process)
        self.webdriverpath = webdriverpath
        self.path_save = path_save
        
    def multi_processing(self, sites, repeat):
        def chat_crawler(url):
            # 사용하는 Library Import
            from selenium import webdriver
            from selenium.webdriver.common.by import By
            import time
            import pandas as pd
            from bs4 import BeautifulSoup

            # 원하는 사이트 로드
            driver = webdriver.Chrome(self.webdriverpath)
            driver.get(url)
            save_number = str(url)
            save_number = save_number[44:50]
            
            # 1분 대기 ... 여러 사이트 열어서 시간이 좀 걸림.
            time.sleep(6*self.n_process)

            # 자동으로 2배속 설정.
            buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
            buttun.click()
            time.sleep(1)
            buttun.click()
            
            # 반복 횟수 설정.
            repeat_n = repeat
            empty = []
            df = pd.DataFrame({'name' : empty, 'chat' : empty})
            
            # 채팅 및 아이디 크롤링.
            for re_n in range(repeat_n):
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
            
                chat_list_temp = []
                for i in temp_list:
                    for j in i:
                        for k in j:
                            chat = k.select('span')
                            chat = str(chat)
                            chat_list_temp.append(chat)

                name_list_temp = []
                for i in temp_list:
                    for j in i:
                        name = j.select('strong')
                        name = str(name)
                        if 'nickname' in name:
                            name_list_temp.append(name)

                chat_list = []
                for i in chat_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        chat_list.append(i)
                    except:
                        continue
                    
                name_list = []
                for i in name_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        name_list.append(i)
                    except:
                        continue
                df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
                df = pd.concat([df, df_temp], ignore_index=True)
                
                time.sleep(150)

            # df 형성 및 저장.
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(self.path_save + '\{}.csv' .format(save_number), index=False)
            
            driver.quit()
        
        self.pool.map(chat_crawler, sites)

In [ ]:
df_list = os.listdir('D:\Google Drive\내 드라이브\LiveCommerse\Video_df')

for i in df_list:
    df = pd.read_csv('D:\Google Drive\내 드라이브\LiveCommerse\Video_df'+f'\{i}')
    url_list = list(df.Video_url)
    
    url_list_1 = url_list[:5]
    url_list_2 = url_list[5:]
    
    category_name = i[:-4]
    path = f'D:\Google Drive\내 드라이브\LiveCommerse\df_save\saveforder_{category_name}'
    
    try:
        Crawler1 = MyCrawler(len(url_list_1), chrome_driver_path, path)
    except:
        continue
    
    try:
        Crawler2 = MyCrawler(len(url_list_2), chrome_driver_path, path)
    except:
        continue
    
    try:
        Crawler1.multi_processing(url_list_1, 24)
    except:
        continue
    
    try:
        Crawler2.multi_processing(url_list_2, 24)
    except:
        continue

In [ ]:
# 크롤러 할당

num1 = MyCrawler(len(crawlnum1), chrome_driver_path, save_path_iope)
num2 = MyCrawler(len(crawlnum2), chrome_driver_path, save_path_iope)

In [ ]:
# 크롤러 병렬 실행

num1.multi_processing(crawlnum1, 12)
num2.multi_processing(crawlnum2, 12)

In [ ]:
# csv 파일 전체 통합

empty = []
df = pd.DataFrame({'name' : empty, 'chat' : empty})

data_csv = os.listdir(save_path_iope)

for i in data_csv:
    df_temp = pd.read_csv(save_path_iope + f'\{i}')
    df = pd.concat([df, df_temp], ignore_index=True)
    
df.reset_index(drop=True, inplace=True)
df.to_csv(save_path_iope + '\\total.csv', index=False)


In [ ]:
df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')
df.name.nunique()

In [ ]:
# 전체 df 가져와서 구매번호 인증 관련된 채팅 제거.

df = pd.read_csv(r'G:\내 드라이브\LiveCommerse\saveforder_iope\total.csv')

pattern = '\d{16,}'
drop_df = df[~(df['chat'].str.contains(pattern))]
drop_df.reset_index(drop=True, inplace=True)
drop_df

In [ ]:
drop_df.name.nunique()

##### 아래는 연습장임

In [ ]:
# 크롤러 만들기.
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)

In [ ]:
# 자동 2배속

def sppedx2(driver):
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()

In [ ]:
# 자동 크롤링 반복 후 df 형성

def crawler(repeat, savepath):
    repeat_n = repeat
    empty = []
    df = pd.DataFrame({'name' : empty, 'chat' : empty})
    
    for re_n in range(repeat_n):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
        chat_list_temp = []
        for i in temp_list:
            for j in i:
                for k in j:
                    chat = k.select('span')
                    chat = str(chat)
                    chat_list_temp.append(chat)

        name_list_temp = []
        for i in temp_list:
            for j in i:
                name = j.select('strong')
                name = str(name)
                if 'nickname' in name:
                    name_list_temp.append(name)

        chat_list = []
        for i in chat_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                chat_list.append(i)
            except:
                continue
            
        name_list = []
        for i in name_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                name_list.append(i)
            except:
                continue
        df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
        df = pd.concat([df, df_temp], ignore_index=True)
        
        time.sleep(150)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(savepath)

In [ ]:
driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
driver.get(replay_video_list[0])

time.sleep(5)
time.sleep(10)

# 자동 2배속
buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
buttun.click()
time.sleep(1)
buttun.click()


repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))

In [ ]:
def scrape(url):
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    
    time.sleep(5)
    time.sleep(10)

    # 자동 2배속
    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()
    
    time.sleep(10)

In [ ]:
pool = ProcessPool(nodes=10)
pool.map(scrape, replay_video_list)

In [ ]:
class WebCrawler:
    def __init__(self):
        self.pool = ProcessPool(nodes=10)
        
    def open_browser(url):
        from selenium import webdriver
        driver = webdriver.Chrome(r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe')
        driver.get(url)
        
    def multi_processing(self, url_list):
        self.pool.map(open_browser, url_list)


In [ ]:
len(replay_video_list)

In [ ]:
replay_video_list

In [ ]:
# 원하는 사이트 오픈

url_num = 2
driver = webdriver.Chrome(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/chromedriver_mac64/chromedriver')
driver.get(url_list[url_num])

In [ ]:
repeat = 12

empty = []
df = pd.DataFrame({'name': empty, 'chat' : empty})

for re_n in range(repeat):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
    chat_list_temp = []
    for i in temp_list:
        for j in i:
            for k in j:
                chat = k.select('span')
                chat = str(chat)
                chat_list_temp.append(chat)

    name_list_temp = []
    for i in temp_list:
        for j in i:
            name = j.select('strong')
            name = str(name)
            if 'nickname' in name:
                name_list_temp.append(name)

    chat_list = []
    for i in chat_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            chat_list.append(i)
        except:
            continue
        
    name_list = []
    for i in name_list_temp:
        try:
            i = (i.split('<')[1])
            i = (i.split('>')[1])
            name_list.append(i)
        except:
            continue
    
    df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
    df = pd.concat([df, df_temp], ignore_index=True)
    
    print(re_n)
    time.sleep(150)

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv(r'/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/df_save_{}.csv' .format(url_num))